# Multi-Agentes en LangChain 1.0
* Para usar sub-agentes, simplemente llámalos como herramientas desde el agente principal.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calcula la raíz cuadrada de un número"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calcula el cuadrado de un número"""
    return x ** 2


from langchain.agents import create_agent

# crear sub-agentes

subagent_1 = create_agent(
    model='gpt-4o-mini',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-4o-mini',
    tools=[square]
)

from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Llama al sub-agente 1 para calcular la raíz cuadrada de un número"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calcula la raíz cuadrada de {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Llama al sub-agente 2 para calcular el cuadrado de un número"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calcula el cuadrado de {x}")]})
    return response["messages"][-1].content

## Creando el agente principal

main_agent = create_agent(
    model='gpt-4o-mini',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="Eres un asistente útil que puede llamar a sub-agentes para calcular la raíz cuadrada o el cuadrado de un número.")


question = "¿Cuál es la raíz cuadrada de 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

print(response['messages'][-1].content)

## Usando pprint podemos ver que el Agente está llamando al sub-agente

In [ ]:
from pprint import pprint

pprint(response)

## OK. Expliquemos el código anterior en términos sencillos
A continuación se presenta una **explicación sencilla, apta para principiantes, línea por línea** de lo que hace este código de LangChain 1.0.
Explicaré **qué significa cada parte y por qué existe**, sin asumir conocimientos previos de LangChain o agentes.

---

#### 1. Importando herramientas para construir agentes

```python
from langchain.agents import create_agent
from langchain.tools import tool
```

* `create_agent` es una función auxiliar que crea un **agente** de IA (un modelo que puede razonar y usar herramientas).
* `tool` es un decorador que convierte una función Python normal en una **herramienta** que el agente puede llamar.

---

#### 2. Creando herramientas matemáticas simples

**Herramienta de raíz cuadrada**

```python
@tool
def square_root(x: float) -> float:
    """Calcula la raíz cuadrada de un número"""
    return x ** 0.5
```

Qué está pasando aquí:

* `@tool` le dice a LangChain:

  > "Esta función puede ser usada por un agente de IA."
* La función:

  * Toma un número (`x`)
  * Devuelve su raíz cuadrada usando `x ** 0.5`
* El docstring ayuda a la IA a entender **qué hace la herramienta**

---

**Herramienta de cuadrado**

```python
@tool
def square(x: float) -> float:
    """Calcula el cuadrado de un número"""
    return x ** 2
```

Misma idea que antes:

* Esta herramienta:

  * Toma un número
  * Devuelve su cuadrado (`x * x`)

---

#### 3. Creando sub-agentes

```python
from langchain.agents import create_agent
```

Esta importación está repetida (no es perjudicial, solo redundante).

---

**Sub-agente 1: agente de raíz cuadrada**

```python
subagent_1 = create_agent(
    model='gpt-4o-mini',
    tools=[square_root]
)
```

Esto crea un agente de IA que:

* Usa el modelo **gpt-4o-mini**
* Tiene acceso a **solo una herramienta**: `square_root`

Piensa en ello como:

> "Una IA que solo sabe cómo calcular raíces cuadradas."

---

**Sub-agente 2: agente de cuadrado**

```python
subagent_2 = create_agent(
    model='gpt-4o-mini',
    tools=[square]
)
```

Este agente:

* Usa el mismo modelo
* Solo puede calcular **cuadrados**

Así que ahora tienes **dos ayudantes de IA especializados**.

---

#### 4. Importando formato de mensaje

```python
from langchain.messages import HumanMessage
```

* `HumanMessage` representa un mensaje escrito por un usuario.
* Los agentes en LangChain se comunican usando objetos de mensaje, no cadenas de texto simples.

---

#### 5. Herramientas que llaman a sub-agentes

**Herramienta para llamar al sub-agente 1 (raíz cuadrada)**

```python
@tool
def call_subagent_1(x: float) -> float:
    """Llama al sub-agente 1 para calcular la raíz cuadrada de un número"""
```

* Esta función en sí también es una **herramienta**
* No hace matemáticas directamente
* En su lugar, **le pide a otro agente que lo haga**

---

```python
response = subagent_1.invoke({
    "messages": [
        HumanMessage(content=f"Calcula la raíz cuadrada de {x}")
    ]
})
```

Qué hace esto:

* Envía un mensaje a `subagent_1`
* El mensaje parece algo que un humano diría
* El agente decide cómo responder (normalmente llamando a `square_root`)

---

```python
return response["messages"][-1].content
```

* Se devuelve la conversación completa del agente
* `[-1]` significa "el último mensaje"
* `.content` extrae el texto de la respuesta real

---

**Herramienta para llamar al sub-agente 2 (cuadrado)**

```python
@tool
def call_subagent_2(x: float) -> float:
    """Llama al sub-agente 2 para calcular el cuadrado de un número"""
```

Esto funciona exactamente igual que `call_subagent_1`, pero:

* Habla con `subagent_2`
* Pide el **cuadrado**, no la raíz cuadrada

---

#### 6. Creando el agente principal

```python
main_agent = create_agent(
    model='gpt-4o-mini',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="Eres un asistente útil que puede llamar a sub-agentes para calcular la raíz cuadrada o el cuadrado de un número."
)
```

Este es el **agente jefe**:

* Usa el mismo modelo de IA
* Tiene acceso a:

  * `call_subagent_1`
  * `call_subagent_2`
* El `system_prompt` le dice:

  > "No calculas directamente. Delegas el trabajo a sub-agentes."

---

#### 7. Haciendo una pregunta

```python
question = "¿Cuál es la raíz cuadrada de 456?"
```

Esta es una pregunta normal de usuario.

---

```python
response = main_agent.invoke({
    "messages": [HumanMessage(content=question)]
})
```

Qué sucede internamente:

1. El agente principal lee la pregunta
2. Decide:

   * "Esto es un problema de raíz cuadrada"
3. Llama a `call_subagent_1`
4. Ese sub-agente llama a la herramienta `square_root`
5. El resultado fluye de vuelta hacia arriba

---

#### 8. Imprimiendo la respuesta final

```python
print(response['messages'][-1].content)
```

* Extrae el mensaje final del agente
* Imprime la respuesta en la consola

---

#### 🧠 Resumen general

* Has creado **herramientas matemáticas pequeñas**
* Las has envuelto en **sub-agentes especializados**
* Has creado un **agente principal** que:

  * Entiende preguntas
  * Delega el trabajo al sub-agente correcto
* Esto imita un **equipo de IAs**, cada una con un rol claro

## Cómo ejecutar este código desde Visual Studio Code
* Abre la Terminal.
* Asegúrate de estar en la carpeta del proyecto.
* Asegúrate de tener el entorno de poetry activado.
* Introduce y ejecuta el siguiente comando:
    * `python 009-multi-agents.py`